### Twilight Struggle

In [1]:
import numpy as np
import random

from twilight_map import *
from game_mechanics import *
from twilight_cards import *

### Game Instance

In [2]:
# this is a temporary measure. It should go into UI.new()
g = Game()
g.ui.run()

Initalising game..

----- USSR Player: -----
You may place 6 influence in these countries. Type in their country indices, separated by commas (no spaces).
East_Germany, 16
Romania, 21
Yugoslavia, 20
Finland, 7
Czechoslovakia, 18
Hungary, 19
Austria, 14
Bulgaria, 22
Poland, 17
> 7,7,7,7,7,7
0 {'7'} {'18', '20', '14', '19', '16', '22', '7', '17', '21'}

----- US Player: -----
You may place 7 influence in these countries. Type in their country indices, separated by commas (no spaces).
Spain_Portugal, 11
Finland, 7
Italy, 12
Sweden, 6
Denmark, 8
West_Germany, 15
France, 10
Turkey, 23
Canada, 3
Austria, 14
Benelux, 9
Norway, 5
Greece, 13
UK, 4
> 4,4,4,4,4,4,4
0 {'4'} {'3', '5', '4', '9', '14', '8', '23', '11', '6', '12', '15', '13', '7', '10'}

----- US Player: -----
You may place 2 influence in these countries. Type in their country indices, separated by commas (no spaces).
US, 2
Canada, 3
UK, 4
Israel, 26
Iran, 30
Australia, 42
Philippines, 44
Japan, 45
South_Korea, 47
South_Africa, 59
Pa